
# 🔐 Public–Private Key Cryptography Lab (with Diffie–Hellman)

Welcome to the **Public–Private Key Cryptography** lab. This notebook is designed for **Google Colab** (or local Jupyter)
and demonstrates the core ideas behind **asymmetric cryptography**, including:

- **Diffie–Hellman (DH) key exchange** (classical and secure-sized demos)
- **RSA**: toy keygen, encrypt/decrypt, sign/verify (for learning — *not secure*)
- **Digital signatures** and why they matter
- **How HTTPS/TLS uses these building blocks** to secure the web
- Strengths, pitfalls, and hands-on exercises

> ⚠️ **Security note**: The RSA code here is intentionally simple for learning. Do **not** use it for real security.


## 0) Setup & Math Helpers

In [ ]:

import secrets
import math
from typing import Tuple

def egcd(a, b):
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    return (g, y1, x1 - (a // b) * y1)

def modinv(a, m):
    g, x, _ = egcd(a, m)
    if g != 1:
        raise ValueError("No modular inverse")
    return x % m

def is_probable_prime(n, k=10):
    if n < 2:
        return False
    small_primes = [2,3,5,7,11,13,17,19,23,29]
    if n in small_primes:
        return True
    if any(n % p == 0 for p in small_primes):
        return False
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    for _ in range(k):
        a = secrets.randbelow(n - 3) + 2
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def gen_prime(bits=512):
    while True:
        candidate = secrets.randbits(bits) | 1 | (1 << (bits-1))
        if is_probable_prime(candidate):
            return candidate

print("✔️ Math helpers loaded.")



### 1) Diffie–Hellman (DH) Key Exchange

Two parties (Alice and Bob) agree on a public prime modulus \( p \) and generator \( g \).
They keep private exponents \( a \) and \( b \), and compute public values:

$$
A = g^a \mod p, \qquad B = g^b \mod p
$$

They exchange \( A \) and \( B \). Each computes the shared secret:

$$
s = B^a \mod p = A^b \mod p = g^{ab} \mod p
$$

An eavesdropper sees \( p, g, A, B \) but—assuming the \textit{Discrete Logarithm Problem} is hard—cannot efficiently recover \( a, b, \) or \( s \).



### 🔐 Diffie–Hellman Example (with real numbers)

**Public values**

$$
p = 23, \qquad g = 5
$$

**Private secrets**

$$
a = 6 \ (\text{Alice}), \qquad b = 15 \ (\text{Bob})
$$

---

#### Step 1 — Compute public keys

$$
A = g^a \mod p = 5^6 \mod 23 = 8
$$

$$
B = g^b \mod p = 5^{15} \mod 23 = 19
$$

---

#### Step 2 — Compute shared secret

$$
s_A = B^a \mod p = 19^6 \mod 23 = 2
$$

$$
s_B = A^b \mod p = 8^{15} \mod 23 = 2
$$

---

#### ✅ Result

$$
\boxed{s_A = s_B = 2}
$$

Even though an eavesdropper sees \(p=23\), \(g=5\), \(A=8\), and \(B=19\),
they cannot easily recover \(a\), \(b\), or \(s\) (for large values) because of the **Discrete Logarithm Problem**.


### 1.a) Tiny demo (not secure, just to see the math)

In [ ]:

p = 23
g = 5

alice_a = 6
bob_b = 15

A = pow(g, alice_a, p)
B = pow(g, bob_b, p)

s_alice = pow(B, alice_a, p)
s_bob   = pow(A, bob_b, p)

print(f"Public params (p={p}, g={g})")
print(f"Alice sends A={A}; Bob sends B={B}")
print(f"Alice shared secret: {s_alice}")
print(f"Bob   shared secret: {s_bob}")
assert s_alice == s_bob


### 1.b) Larger random demo (still for teaching; not production)

In [ ]:

p = gen_prime(bits=256)
g = 2

alice_a = secrets.randbits(200)
bob_b   = secrets.randbits(200)

A = pow(g, alice_a, p)
B = pow(g, bob_b, p)

s_alice = pow(B, alice_a, p)
s_bob   = pow(A, bob_b, p)

print("p (bits):", p.bit_length())
print("g:", g)
print("A (Alice pub):", A)
print("B (Bob pub):", B)
print("Shared secret: ", s_alice)
print("Shared secret matches?", s_alice == s_bob)



## 2) RSA (Toy Implementation for Learning)

RSA relies on the difficulty of **integer factorization**.

1. Choose two large primes \( p, q \), compute \( n = pq \) and \( \phi(n) = (p-1)(q-1) \).
2. Pick public exponent \( e \) coprime to \( \phi(n) \) (commonly 65537).
3. Compute private exponent \( d = e^{-1} \bmod \phi(n) \).
4. **Public key**: \( (n, e) \). **Private key**: \( d \) (and \( p, q \)).

- **Encrypt**: \( c = m^e \bmod n \)
- **Decrypt**: \( m = c^d \bmod n \)

> This demo omits padding (e.g., OAEP) and real-world safeguards — it’s strictly educational.


### 2.a) RSA key generation, encrypt/decrypt

In [ ]:

import math

def rsa_keygen(bits=512):
    p = gen_prime(bits // 2)
    q = gen_prime(bits // 2)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    if math.gcd(e, phi) != 1:
        return rsa_keygen(bits)
    d = modinv(e, phi)
    return (n, e, d, p, q)

def rsa_encrypt(m_int, n, e):
    if m_int >= n:
        raise ValueError("Message too large for modulus (no padding used).")
    return pow(m_int, e, n)

def rsa_decrypt(c_int, n, d):
    return pow(c_int, d, n)

n, e, d, p, q = rsa_keygen(bits=512)
message = b"HELLO INTERNET"
m_int = int.from_bytes(message, "big")

c_int = rsa_encrypt(m_int, n, e)
m2_int = rsa_decrypt(c_int, n, d)
decoded = m2_int.to_bytes((m2_int.bit_length()+7)//8, "big")

print("RSA modulus bits:", n.bit_length())
print("RSA modulus value:", n)
print("RSA public exponent bits:", e.bit_length())
print("RSA public exponent value:", e)
print("Cipher integer:", c_int)
print("Recovered:", decoded)


### 2.b) RSA digital signature (toy)

In [ ]:

import hashlib

def rsa_sign(message_bytes, n, d):
    h = hashlib.sha256(message_bytes).digest()
    h_int = int.from_bytes(h, "big")
    return pow(h_int, d, n)

def rsa_verify(message_bytes, signature_int, n, e):
    h = hashlib.sha256(message_bytes).digest()
    h_int = int.from_bytes(h, "big")
    check = pow(signature_int, e, n)
    return check == h_int

msg = b"I certify that this is authentic."
sig = rsa_sign(msg, n, d)
print("Signature integer:", sig)
print("Verify? ->", rsa_verify(msg, sig, n, e))
print("Verify altered? ->", rsa_verify(msg + b"!", sig, n, e))



## 3) Why the Web is Secure: Hybrid Crypto & TLS

Modern **TLS (HTTPS)** uses a **hybrid** approach:

- Use **public–private key crypto** (RSA or ECDSA for certificates, plus **ephemeral Diffie–Hellman** (ECDHE/DHE)) to **authenticate** the server and **establish a shared secret**.
- Derive **symmetric session keys** (AES/ChaCha20) from the shared secret to **encrypt** bulk data efficiently.
- **Certificates** are signed by trusted **Certificate Authorities (CAs)** so your browser can verify you’re talking to the real website (authenticity).

### High-level TLS 1.2/1.3 flow (simplified)
1. Client says hello (ciphers it supports, random nonce).
2. Server responds with certificate (public key) and parameters for **ephemeral DH** (e.g., ECDHE).
3. Both sides compute a shared secret via DH, derive symmetric keys.
4. Client verifies server certificate (chain to a trusted CA), checks hostname.
5. Switch to symmetric encryption; all application data (HTTP) is now encrypted and integrity-protected.

**Strengths enabled:** confidentiality, integrity, authentication, forward secrecy (with ephemeral DH).



## 4) Strengths and Pitfalls

**Strengths**
- **Confidentiality**: Eavesdroppers can’t read encrypted traffic.
- **Integrity**: Tampering is detected.
- **Authentication**: Certificates ensure you’re connected to the right site.
- **Forward Secrecy (FS)**: Ephemeral DH means past sessions stay safe even if long-term keys leak later.

**Pitfalls / Cautions**
- **Toy crypto ≠ real crypto**: Never roll your own; use vetted libraries (e.g., libsodium, OpenSSL, cryptography).
- **Key management** is hard: protect private keys, rotate periodically, use HSMs where appropriate.
- **Implementation bugs** and weak randomness break security.
- **Algorithm choice**: Prefer modern suites (TLS 1.3, ECDHE, AES-GCM/ChaCha20-Poly1305, RSA-PSS/ECDSA).



## 5) Bonus: Use DH to derive a symmetric key and encrypt a message

We’ll do a quick (educational) symmetric encryption with Python’s `cryptography` package **if available**.
If not, we’ll fall back to a minimal XOR stream (for concept only). In production, always use vetted libraries.


In [ ]:

import hashlib, os

# Recompute a DH exchange for this section
def demo_dh(bits=192):
    p_demo = gen_prime(bits=bits)
    g_demo = 5
    a_demo = secrets.randbits(bits-32)
    b_demo = secrets.randbits(bits-32)
    A_demo = pow(g_demo, a_demo, p_demo)
    B_demo = pow(g_demo, b_demo, p_demo)
    sA = pow(B_demo, a_demo, p_demo)
    sB = pow(A_demo, b_demo, p_demo)
    assert sA == sB
    return p_demo, g_demo, A_demo, B_demo, sA

p_demo, g_demo, A_demo, B_demo, sA = demo_dh(bits=192)
shared_bytes = sA.to_bytes((sA.bit_length()+7)//8, "big")
key = hashlib.sha256(shared_bytes).digest()

plaintext = b"Meet at 9pm. Bring pizza."
print("Derived key (first 16 bytes):", key[:16])

try:
    from cryptography.fernet import Fernet
    import base64
    fkey = base64.urlsafe_b64encode(key)
    f = Fernet(fkey)
    ct = f.encrypt(plaintext)
    pt = f.decrypt(ct)
    print("Fernet available -> ciphertext length:", len(ct))
    print("Recovered:", pt)
except Exception as e:
    stream = hashlib.sha256(key + b"nonce").digest()
    ct = bytes([p ^ stream[i % len(stream)] for i, p in enumerate(plaintext)])
    pt = bytes([c ^ stream[i % len(stream)] for i, c in enumerate(ct)])
    print("Fernet unavailable; using XOR demo. Ciphertext:", ct)
    print("Recovered:", pt)



## 6) Exercises

1. **DH Variations**: Change \( p, g \) and private exponents. Confirm both sides still derive the same secret.
2. **Eavesdropper Challenge**: Given \( p, g, A, B \) from the tiny demo, try to recover the shared secret without \( a \) or \( b \). Why is this hard for large \( p \)?
3. **RSA Limits**: Try encrypting a message that is too large for the modulus. What happens? How do real systems handle arbitrary-length data (hint: hybrid encryption & padding)?
4. **Signatures**: Modify the message after signing and verify that validation fails. Explain how signatures provide integrity and authenticity.
5. **TLS Thought Experiment**: Sketch the TLS 1.3 handshake message flow and label where DH, certificates, and symmetric keys appear.
6. **Forward Secrecy**: Explain why using new ephemeral DH keys for each session protects past conversations if a server’s private key leaks later.



## Appendix: TLS Key Ideas in One Screen

- **Authentication**: Server proves identity with a CA-signed certificate (RSA/ECDSA public key).
- **Key Exchange**: Ephemeral **(EC)DHE** performs DH to agree on a shared secret.
- **Key Derivation**: HKDF expands the shared secret into multiple traffic keys.
- **Symmetric Encryption**: AES-GCM or ChaCha20-Poly1305 protects data (confidentiality & integrity).
- **Certificates & Trust**: Browsers trust a root store; intermediates sign server certs; clients verify chain + hostname + validity.
